In [3]:
# export openblas_num_threads=1; export OMP_NUM_THREADS=1; python simu.py

from utils import * 
from weight import * 
import simu_funs
from main import *


## Initialize Env

In [ ]:
a = now()
##########################################################################################################################################################

rep_times = 100
region_parallel = False
full_parallel = False
pattern_seed = 2
sd_u_O = 25
w_O = .5
w_A = 1.5
u_D_range = [80] 
thre_range = [100, 101, 105, 110]

penalty_range = [[3e-4, 1e-4, 5e-5], [3e-4, 1e-4, 5e-5]]
with_IS = True
n_layer = 3
max_iteration = 1001
Learning_rate = 5e-4
w_hidden = 30
batch_size = 32

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
tf.keras.backend.set_floatx('float64')

import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"



In [5]:
with_NO_MARL = False
with_MF = False

l = 5
t_func = None
sd_D = None
sd_O = None
CV_QV = True
penalty_NMF = [[1e-3], [1e-3]]
dim_S_plus_Ts = 3 + 3
epsilon = 1e-6
penalty = penalty_range
inner_parallel = region_parallel

n_gpu = 8



In [6]:
aim = "final_sd"

sd_R_range = [0, 5, 10, 15, 20, 25, 30]
day_range = [7]
print("[thre_range, sd_R_range, day_range, penalty_range]: ", [thre_range, sd_R_range, day_range, penalty_range])
results, res_real = [], []

u_D = 80
day = 7
T = day * 48

for sd_R in sd_R_range: # vary
    print_setting = DASH + "[pattern_seed, day, sd_R, u_D] = " + str([pattern_seed, day, sd_R, u_D]) + "\n"
    print(print_setting)
    
    ####################################################################################################################################################################################
    np.random.seed(pattern_seed)
    u_O = rnorm(100, sd_u_O, l**2) 

    print("max(u_O) = ", np.round(max(u_O), 1), "mean(u_O) = ", np.round(np.mean(u_O), 1))

    # generate the corresponding target plicy
    target_policys = []
    n_tp = len(thre_range)
    for i in range(n_tp):
        O_thre = thre_range[i]
        print("O_threshold = " + str(thre_range[i]))
        if i == 0: 
            target_policy = simu_DGP.simu_target_policy_pattern(l = l, u_O = u_O, threshold =  O_thre, print_flag = "all") # "all"
        else:
            target_policy = simu_DGP.simu_target_policy_pattern(l = l, u_O = u_O, threshold =  O_thre, print_flag = "None") # "policy_only"
        target_policys.append(target_policy)

    # generate the adj for the grid
    neigh = adj2neigh(getAdjGrid(l))
    break

[thre_range, sd_R_range, day_range, penalty_range]:  [[100, 101, 105, 110], [0, 5, 10, 15, 20, 25, 30], [7], [[0.0003, 0.0001, 5e-05], [0.0003, 0.0001, 5e-05]]]

--------------------------------------
[pattern_seed, day, sd_R, u_D] = [2, 7, 0, 80]

max(u_O) =  157.3 mean(u_O) =  93.7
O_threshold = 100
means of Order: 

89.6 98.6 46.6 141.0 55.2 

79.0 112.6 68.9 73.6 77.3 

113.8 157.3 101.0 72.1 113.5 

85.1 99.5 129.4 81.3 100.2 

78.0 96.1 106.4 75.3 91.5 

target policy: 

0 0 0 1 0 

0 1 0 0 0 

1 1 1 0 1 

0 0 1 0 1 

0 0 1 0 0 

number of reward locations:  9
O_threshold = 101
number of reward locations:  8
O_threshold = 105
number of reward locations:  7
O_threshold = 110
number of reward locations:  6


## Simulate Data (for a given region i)

In [20]:
# reg_data = n_region * n_days * [12 components]
# [S_it, A_it, R_it, Tsit, Tait,  # 0 - 4
#                     S_i(t+1), Tsi(t+1), pi_Sit_1, T_ait_1_pi, # 5 - 8 : pi_Sit_1 = \pi(S_{i, t + 1}), T_ait_1_pi = T_{a, i, t + 1}(\pi)
#                     A_it1, pi_Sit, T_ait_pi] # 9 - 11
data_test_reps = []
for seed in tqdm(range(100)):
    reload(simu_funs)
    data, adj_mat, details, reg_data = simu_funs.simu_once(seed = seed, l = l, T = T, t_func = t_func,  u_D = u_D,
                             u_O = u_O,  
                             target_policys = target_policys, w_A = w_A, w_O = w_O, dim_S_plus_Ts = dim_S_plus_Ts,  
                              penalty = penalty, penalty_NMF = penalty_NMF, 
                             n_layer = n_layer, sd_D = sd_D, sd_R = sd_R, sd_O = sd_O, 
                              w_hidden = w_hidden, Learning_rate = Learning_rate,  CV_QV = CV_QV, 
                              batch_size = batch_size, max_iteration = max_iteration, epsilon = epsilon, 
                             with_MF = with_MF, with_NO_MARL = with_NO_MARL, with_IS = with_IS, 
                               inner_parallel = inner_parallel, gpu_number = seed % 8
                                , return_datail_only = True)
    i = 1 # region id
    X = []
    A = []
    R = []
    for t in range(len(reg_data[i])):
        s = np.concatenate([reg_data[i][t][0], reg_data[i][t][3], [reg_data[i][t][4]]])
        a = reg_data[i][t][1]
        r = reg_data[i][t][2]
        X.append(s)
        R.append(r)
        A.append(a)
    X = arr(X)
    A = arr(A)
    R = arr(R)
    R = np.expand_dims(R, 1)
    A = np.expand_dims(A, 1)
    if include_reward:
        data_test_reps.append([[X[:(335 // 2)], A[:(335 // 2)], R[:(335 // 2)]]
                           , [X[(335 // 2 + 1):], A[(335 // 2 + 1):], R[(335 // 2 + 1):]]])
    else:
        data_test_reps.append([[X[:(335 // 2)], A[:(335 // 2)]], [X[(335 // 2 + 1):], A[(335 // 2 + 1):]]])

  0%|          | 0/100 [00:00<?, ?it/s]

seed = 0


  1%|          | 1/100 [00:00<01:18,  1.26it/s]

seed = 1


  2%|▏         | 2/100 [00:01<01:18,  1.25it/s]

...

 98%|█████████▊| 98/100 [01:20<00:01,  1.25it/s]

seed = 98


 99%|█████████▉| 99/100 [01:21<00:00,  1.25it/s]

seed = 99


100%|██████████| 100/100 [01:22<00:00,  1.21it/s]


## Run the test

In [15]:
include_reward = False

In [11]:
import _core_test_fun
reload(_core_test_fun)
def get_one_p_value(seed):
    test1 = _core_test_fun.MarkovTester(data_test_reps[seed], J = 1,
            B = 100, Q = 2, L = 2, 
            paras= [3, 10], n_trees = 50, 
            print_time = False,
            include_reward = include_reward)
    return test1.pValues